# <i>ExpressNet</i>: Teach The Machine To Write The Daily Express

This very organised and well-commented notebook is based heavily on [this article](#https://www.analyticsvidhya.com/blog/2018/03/text-generation-using-python-nlp/), except it's been adapted to do words instead of characters (which I think is legit because the number of unique words is *fairly* low but it might not be legit).

First we download the data using BeautifulSoup

In [5]:
import numpy as np
import datetime as dt
import xarray as xa
import urllib.request
from bs4 import BeautifulSoup
import re
from collections import Counter
import string
### Gets all headlines

days = np.arange('2020-10-12', '2020-11-12', dtype='datetime64[D]')
days = np.arange('2015-01-01', '2020-11-18', dtype='datetime64[D]')
# days = np.arange('2016-07-17', '2020-11-18', dtype='datetime64[D]')

headlines = []
newsdays = []
for d in range(len(days)):
    day = int(str(days[d])[-2:])
    month = int(str(days[d])[5:7])
    year = int(str(days[d])[:4])
    url = 'https://www.express.co.uk/sitearchive/' + str(year) + '/' + str(month) + '/' + str(day)
    print('Searching ' + url)
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    headline_links_section = soup.find("ul","section-list")
    weather_links = headline_links_section.find_all(href=re.compile("/weather"))
    for link in weather_links:
        headline = str(link.find("span"))[6:-7]
        headlines.append(headline)
        newsdays.append(str(year) + '-' + str(month) + '-' + str(day))
print(headlines)
print(newsdays)
# print(soup.prettify())
# print(headline_links.prettify())

The cell below can be used to save headlines into .txt files and then open them again.

In [2]:
## Save the headlines
# headlines_str = str(headlines)
# with open("headlines_from_2015.txt", "w") as file:
#     file.write(headlines_str)

##Read the headlines
with open("headlines_from_2015.txt", "r") as file:
    headlines_adj2 = eval(file.readline())


In [15]:
len(headlines)/len(set(newsdays))  ##number of headlines per day 

5.679431747992588

The following extracts every word that is capitalised. This includes stuff like "24C" etc.

In [3]:
capitalised_words = []

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

acronyms = ['UK','US','BBC','NOAA'] #this is the list of acronyms to exclude from the capital count
capsperheadline = []
for h in headlines_adj2:
    words = h.split()
    #only capitals
    capitals = [word for word in words if word.isupper()]
    #removes punctuation
    capitals = [word.translate(str.maketrans('','',string.punctuation)) for word in capitals]
    #removes temperatures
    capitals = [word for word in capitals if hasNumbers(word)==False]
    #removes acronyms
    capitals = [word for word in capitals if word not in acronyms]
    capitalised_words.extend(capitals)
# print(capitalised_words)
capital_dict = dict(Counter(capitalised_words))
# print(capital_dict)

In [16]:
## calculates number of capitalised words
numbercaps = []
for i in capital_dict.values():
    numbercaps.append(i)
    
np.sum(numbercaps)

12655

In [17]:
all_words = []
for i in headlines_adj2:
    words = i.split()
    for j in words:
        all_words.append(j)
    
print(len(all_words))
print(len(set(all_words)))
print(len(set(all_words))/len(all_words))
print(all_words)

125266
8673
0.06923666437820318
['What', 'is', 'thundersnow?', 'Britain', 'prepares', 'for', 'Arctic', 'gales', 'and', 'blizzards', 'this', 'week', 'Tips', 'to', 'help', 'your', 'home', 'weather', 'the', 'coming', 'storms', 'Goodbye', 'heatwave:', 'Wild', 'storms', 'set', 'to', 'bash', 'Britain', 'this', 'week', 'Britain', 'to', 'be', 'lashed', 'by', 'torrential', 'rain,', 'gales', 'and', 'floods', 'TONIGHT', 'Five', 'rescued', 'from', 'sinking', 'yacht', 'after', 'major', 'race-against-time', 'helicopter', 'search', 'drama', 'Storm', 'warning', 'across', 'Britain', 'as', 'rising', 'temperatures', 'will', 'trigger', 'thunderstorms', 'WEATHER', 'LATEST:', 'Thunderstorms', 'sweep', 'Britain', 'as', 'MONTHS', 'worth', 'of', 'rain', 'falls', 'in', '9', 'HOURS', 'STORM', 'WATCH:', 'Grim', 'outlook', 'puts', 'dampers', 'on', 'return', 'of', 'summer', 'weather', 'Here', 'comes', 'the', 'TORNADO:', 'Extreme', 'weather', 'warning', 'as', 'UK', 'set', 'for', '84F', 'tropical', 'storms', 'TODAY',

## The neural net bit

This gets a list of ALL the words.

In [23]:
##importing More Stuff for neural net purposes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils


Using TensorFlow backend.


In [20]:
#defines all the 
unique_words = sorted(list(set(all_words)))
n_to_word = {n:word for n, word in enumerate(unique_words)}
word_to_n = {word:n for n, word in enumerate(unique_words)}


In [59]:
print(headlines_adj2) ## sometimes i just like to look through these

['What is thundersnow? Britain prepares for Arctic gales and blizzards this week', 'Tips to help your home weather the coming storms', 'Goodbye heatwave: Wild storms set to bash Britain this week', 'Britain to be lashed by torrential rain, gales and floods TONIGHT', 'Five rescued from sinking yacht after major race-against-time helicopter search drama', 'Storm warning across Britain as rising temperatures will trigger thunderstorms', 'WEATHER LATEST: Thunderstorms sweep Britain as MONTHS worth of rain falls in 9 HOURS', 'STORM WATCH: Grim outlook puts dampers on return of summer weather', 'Here comes the TORNADO: Extreme weather warning as UK set for 84F tropical storms TODAY', 'Bank Holiday weekend WASHOUT: Britain set for even MORE rain and gales as summer ends', 'Fair weather friends: BBC ditch Met Office but we take a look at our FAVOURITE forecasters', 'BBC fires Met Office after 93 years - but could use a DUTCH weather forecaster instead', 'Rare OCTOPUS CYCLONE to bring torrentia

In [18]:
words_per_headline = []
for j in headlines_adj2:
    words_per_headline.append(len(j.split()))
    
np.mean(words_per_headline)

13.652970027247957

In [21]:
##converts words into integers for mapping

X = []
Y = []
seq_length = 2
for j in headlines_adj2:
    length = len(j.split()) ## maybe add a signifier for ending a headline???? otherwise they keep repeating indefinitely
    for i in range(0, length-seq_length, 1):
        sequence = j.split()[i:i + seq_length]
        label = j.split()[i + seq_length]
        X.append([word_to_n[word] for word in sequence])
        Y.append(word_to_n[label])

In [24]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(unique_words))
Y_modified = np_utils.to_categorical(Y)

In [25]:
## edit number of layers (by adding/removing model.add statements)
## or number of nodes (these are 700 currently)
model = Sequential()
model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [233]:
## edit number of epochs and batch size here. epochs makes it ?? better?? but takes longer. no idea what batch size does tbh
model.fit(X_modified, Y_modified, epochs=100, batch_size=50)

model.save_weights('express_generator_700_0.2_700_0.2_700_0.2_100epoch.h5')

Epoch 1/100
106916/106916 [==============================] - 1935s 18ms/step - loss: 6.7107
Epoch 2/100
106916/106916 [==============================] - 2043s 19ms/step - loss: 6.4149
Epoch 3/100
106916/106916 [==============================] - 1925s 18ms/step - loss: 6.2362
Epoch 4/100
106916/106916 [==============================] - 1681s 16ms/step - loss: 6.1014
Epoch 5/100
106916/106916 [==============================] - 2609s 24ms/step - loss: 5.9861
Epoch 6/100
106916/106916 [==============================] - 1459s 14ms/step - loss: 5.8896
Epoch 7/100
106916/106916 [==============================] - 1386s 13ms/step - loss: 5.8085
Epoch 8/100
106916/106916 [==============================] - 1295s 12ms/step - loss: 5.7375
Epoch 9/100
106916/106916 [==============================] - 1431s 13ms/step - loss: 5.6706
Epoch 10/100
106916/106916 [==============================] - 1343s 13ms/step - loss: 5.6075
Epoch 11/100
106916/106916 [==============================] - 1785s 17ms/step -

In [26]:
## loads previously-run versions of the net
model.load_weights('express_generator_700_0.2_700_0.2_700_0.2_100epoch.h5')


This is a bit of a mess but basically it takes 2 initial words (either from original headlines or just any words that exist that you choose to put together) and generates a headline of length "lomgth" because it still doesnt know how to end things. 

In [29]:
lomgth =15 
import copy
# for k in range(0,59347,168):
for k in [10]:

# for k in headlines_adj2:
#     thing = X[k]
    starter = 'Christmas 2020 '.split()
#     starter = k.split()[0:2]
    thing = [word_to_n[value] for value in starter]
    string_mapped = copy.deepcopy(thing)
    print(string_mapped)
    full_string = [n_to_word[value] for value in string_mapped]
    starting_string = ""
    for word in full_string:
        starting_string = starting_string + word + ' '
#     print('Starting with: ' + starting_string)
    # generating characters
    for i in range(lomngth):
        x = np.reshape(string_mapped,(1,len(string_mapped),1))
        x = x / float(len(unique_words))
        pred_index = np.argmax(model.predict(x, verbose=0))
        seq = [n_to_word[value] for value in string_mapped]
        nxt_word = n_to_word[pred_index]
        full_string.append(nxt_word)
        string_mapped.append(pred_index)
        string_mapped = string_mapped[1:len(string_mapped)]
    txt = ''
    for word in full_string:
        txt = txt + word + ' '
    print(txt)
    print('')
    

[1794, 855]
Christmas 2020 NOAA Office weather of Britain to be Britain with 80mph winds to batter Britain with 

